In [1]:
from __future__ import print_function

from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES, interplevel
import glob,os
from tqdm import tqdm
import numpy as np

### Read files

In [2]:
path='/scratch/06040/tg853394/tc/output/redux/maria/ncrf_60h/'
filelist = sorted(glob.glob(path+'wrfout*'))
print(len(filelist))

85


In [3]:
from wrf import (getvar, ALL_TIMES, extract_vars,
                 omp_set_num_threads, omp_get_num_procs)
wrfin = [Dataset(x) for x in filelist]
#my_cache = extract_vars(wrfin, ALL_TIMES, ("QVAPOR"))

In [4]:
#RTHFTEN,REFL_10CM
import time
def combine_wrfoutVAR(filenamelist=None,varname=None,resultshape=None,times_per_file=1,TYPE="3D"):
    if TYPE=='3D':
        z_final = np.empty(resultshape, np.float32)
        for timeidx in (range(resultshape[0])):
            if timeidx % 20==0:
                print(timeidx)
            #fileidx = timeidx // times_per_file
            file_timeidx = timeidx % times_per_file
            
            z = getvar(wrfin[timeidx], varname, file_timeidx)
            z_final[timeidx,:] = z[:]
        return z_final
omp_set_num_threads(omp_get_num_procs())
Tall = combine_wrfoutVAR(filelist,"RTHFTEN",(len(filelist), 54, 750, 1248),1,"3D")

0
20
40
60
80


In [5]:
pall = combine_wrfoutVAR(filelist,"pressure",(len(filelist), 54, 750, 1248),1,"3D")

0
20
40
60
80


#### Interpolation

In [6]:
import xarray as xr
path_coor = '/scratch/06040/tg853394/tc/output/redux/maria/ctl/post/U.nc'
coor = xr.open_dataset(path_coor)
pres_target = coor.pres.data

In [ ]:
Tall_interp = [interplevel(Tall[i,:,:,:], pall[i,:,:,:], pres_target) for i in range(len(Tall))]

In [ ]:
xlat,xlon = wrfin[0].variables['XLAT'][:],wrfin[0].variables['XLONG'][:]
del wrfin,Tall,pall

resultshape=(len(filelist), 39, 750, 1248)
z_final = np.empty(resultshape, np.float32)
for timeidx in (range(resultshape[0])):
    if timeidx % 20==0:
        print(timeidx)
    z_final[timeidx,:] = Tall_interp[timeidx]

In [ ]:
import pandas as pd
data_set=xr.Dataset( coords={'time': pd.date_range(start='9/17/2017', end='9/21/2017',freq='H')[:-12],
                             'pressure': coor.pres.data,
                             'lat': (['north_south', 'west_east'], np.squeeze(xlat)),
                             'lon': (['north_south', 'west_east'], np.squeeze(xlon))})
data_set["RTHFTEN"]=(['time', 'pressure', 'north_south', 'west_east'],  z_final)

In [ ]:
data_set.to_netcdf(path='./RTHFTENinterp_maria_ncrf60.nc')

In [ ]:
#CTRL: pd.date_range(start='9/14/2017', end='9/21/2017',freq='H')[12:-12]

#import json,pickle
#def save_to_pickle(loc=None,var=None):
#    with open(loc,"wb") as f:
#        pickle.dump(var,f)
#    return None